In [1]:
import requests
from datetime import date
import ryan
import fire

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:
# Use a service account.
cred = credentials.Certificate('flightsAccount.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
#matching destinations
def matchDestinations(Origin, Arrival):
    matchingDestinations = []
    for destination in Origin:
        if destination in Arrival:
            matchingDestinations.append(d)
    return matchingDestinations

In [4]:
#imput:
trip = 'VNO-PSR'
Outbound = '2023-09-04'
Return = '2023-09-24'

In [5]:
#checking for destinations and airports
checkDate = int(str(date.today()).replace('-',''))
for origin in trip.split('-'):
    fireBatch = db.batch()
    #API call for destinations
    destinations = requests.get(ryan.getDestinations(origin,2)).json()
    #read response, set batch for Firestore and create array of destinations 
    rDestinations = ryan.readDestinations(destinations, fireBatch, db)
    #commiting batch
    fireBatch.commit()

    #create or update destinations from airport
    fire.fireDestinations(db, origin, rDestinations)

    #get array of matching destinations
    if origin == trip[:3]:
        rOriginDestinations = rDestinations
    else:
        rArrivalDestinations = rDestinations


#array for matched destinations
rConnections = []
if trip[:3] in rArrivalDestinations:
    isDirect = True
else:
    isDirect = False
for d in rOriginDestinations:
    if d in rArrivalDestinations:
        rConnections.append(d)
    
fire.fireConnections(db, trip, rConnections, isDirect)
fire.fireTrip(db, trip, int(str(date.today()).replace('-','')), True, rConnections, isDirect, Outbound.replace('-',''), Return.replace('-',''))
# fire.fireTrip(db, trip, int(str(date.today()).replace('-','')), True, rConnections, isDirect, Outbound.replace('-',''))

In [6]:
for i in rConnections:
    url = ryan.getFlights(i, trip[:3], Outbound, Return)
    print(url)

https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=BGY&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-04&FlexDaysOut=6&DateIn=2023-09-24&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=HHN&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-04&FlexDaysOut=6&DateIn=2023-09-24&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=MLA&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-04&FlexDaysOut=6&DateIn=2023-09-24&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=STN&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-04&FlexDaysOut=6&DateIn=2023-09-24&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&De

In [42]:
headers = {'Cookie': 'rid=53f81685-dd59-43a6-b829-2ac46e936cfa; rid.sig=OCBtG/WF9HO7H4S44kdr0UpsAs/uDH/Z89AjODYV4ipIZy+AD71pUFlLzf5wzEzBcRyxAYtAFGAtx4qTR8QVkUn51bdpJ6adVE+jcjrjjI+gS4t1gxaDdvBZs4Or2Dqxo/HarouZ+NRwtOYrk1d1WKoAbvkPW4Tr/OxK37O7JzNEQfTj/NYjObt4Vn8oTIij4S7BJ6f24RHOKf3Y2y1Gix27jFbvUe5Mh+5Pg4el5c309TyCtbIvycl9N1tcPeYXv7dDnk4KuGcxzl4zoVtTRXMh8O4JEr7shHjgB995Vpd6nJEJpVdtgYqGP51NK7djMcZB+HdJvDmgVI8556bYQxZle072SBcjGJgHC4k7a2spEA0OeaH1gG1gOzpODL7pubJzExfWNrrV94hudE3SKOY4S3wFdyNs6GB8QHwq/okZ+Xf+OEDZyhNAWDTMyZuzxoNPDME98mfIBDTRCuHeX1/G5ufUCC1G+qf0CBFoNhu8hXYOsfs/Bdh3ZkX92zTS0HeerO8LXCg4C9hpgq+g16tRewd1IdaOXRHmtVdMizw=; _ga=GA1.2.185346392.1675256153; STORAGE_PREFERENCES={"STRICTLY_NECESSARY":true,"PERFORMANCE":true,"FUNCTIONAL":true,"TARGETING":true,"SOCIAL_MEDIA":true,"PIXEL":true,"GANALYTICS":true,"__VERSION":2}; RY_COOKIE_CONSENT=true; bid_FRwdAp7a9G2cnLnTsgyBNeduseKcPcRy=8d7200bc-fe65-4a35-828c-05c5db690b69; _hjSessionUser_135144=eyJpZCI6ImYwNTcyYWYyLTVhMDYtNTg0My1hNzBkLWNjMTNjOTM5M2M4NiIsImNyZWF0ZWQiOjE2NzUyNTYxNTY1NzUsImV4aXN0aW5nIjp0cnVlfQ==; agsd=BXFqu0zsEJw9FQlrZw94l5ocpHvV_o6gxW4zfcOak1tJRf33; _cc=AcNukn4KxyCeXwwCm54JSS4K; fr-correlation-id=3c07d844-3f72-46a1-aaea-d81c822ba5e2; fr-correlation-id.sig=NNzxo5fz1Lzxa66hZof7b0c5szc; tj_seed=0025e97ccc74a598f297a6b2bdaf000000; __zlcmid=1F9liBTdl5vMDMU; _hjSessionUser_2481828=eyJpZCI6IjFhZjg1ODI5LTkwZjUtNTA3ZC1iNTVhLTg2YzJkNjkzNDViNiIsImNyZWF0ZWQiOjE2ODAyNTcxNzY5NjEsImV4aXN0aW5nIjpmYWxzZX0=; mkt=/gb/en/; _gid=GA1.2.161670609.1685688242; myRyanairID=; .AspNetCore.Session=CfDJ8Cxd1z5BGy5NmH8fAYZlb1QxQ80qqUfxxfQ0Ce7BOk%2FNZ7BhISvihHFiWzKXZvIgJG%2FDmB1Jjuo8gU9so9BmOcma7t3zeVdF%2FISPfGtsYYO96Jq8umNDC9mKclqpp%2BPqQbnlCl2OvfIUtOl7XAyfhQOC93VWLZ1Mno9lZYV8s%2BLm',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

ryanFlight = requests.get(ryan.getFlights('BGY', trip[:3], Outbound, Return), headers=headers).json()

In [83]:
for f in ryanFlight['trips']:
    print(f"{f['origin']}-{f['destination']}")
    for d in f['dates']:
        if d['flights'] != []:
            print(d['dateOut'], d['flights'])

VNO-BGY
2023-09-04T00:00:00.000 [{'faresLeft': -1, 'flightKey': 'FR~2871~ ~~VNO~09/04/2023 07:30~BGY~09/04/2023 09:05~~', 'infantsLeft': 18, 'regularFare': {'fareKey': 'C2MCNHIDTOISVRVUREIXW724TURS65ND2E3N5PMRJJHPORCDVZGDDARZWYMGNPTERO6NXSIZL73OHUIQEXQJ2EOYBVWGRIVV7L55EYG5OYABVSJ3M572D3OYHI5CARBMWB2LMHSKRGORY43Q3NBTBU4QOXKSFUELW4H3PJWYWYOQSLEAVGK47ZEYLOFB7WLRMX7VPP46ONEACBI3OGOH4S2TBRTP5CEX66ZKOPIL53YG352GGQFBXUOR63WVH7BJ', 'fareClass': 'E', 'fares': [{'type': 'ADT', 'amount': 123.06, 'count': 1, 'hasDiscount': False, 'publishedFare': 123.06, 'discountInPercent': 0, 'hasPromoDiscount': False, 'discountAmount': 0.0, 'hasBogof': False}]}, 'operatedBy': 'Buzz', 'segments': [{'segmentNr': 0, 'origin': 'VNO', 'destination': 'BGY', 'flightNumber': 'FR 2871', 'time': ['2023-09-04T07:30:00.000', '2023-09-04T09:05:00.000'], 'timeUTC': ['2023-09-04T04:30:00.000Z', '2023-09-04T07:05:00.000Z'], 'duration': '02:35'}], 'flightNumber': 'FR 2871', 'time': ['2023-09-04T07:30:00.000', '2023-09-04T09:05: